In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests

En una primera instancia, importamos la API del Estado Nacional argentino que nos posibilitará realizar la normalización de las unidades territoriales y coordenadas

In [2]:
def georef_normalize_entities(data, entity, fields, params=None, prefix='gr_', step_size=1000):
    if not params:
        params = {}

    normalized = pd.DataFrame()

    # Partir el DataFrame de entrada en partes de 'step_size' de longitud
    for i in range(0, len(data), step_size):
        end = min(len(data), i + step_size)
        queries = []
        
        for j in range(i, end):
            # Se crea cada 'consulta' individual (equivalente a hacer un request GET), y
            # se la agrega a una lista de consultas.
            query = {
                'max': 1,
                'aplanar': True,
                **params
            }

            for key, value in fields.items():
                query[key] = str(data.iloc[j][value])

            queries.append(query)

        body = {
            entity: queries
        }

        url = 'https://apis.datos.gob.ar/georef/api/{}'.format(entity)

        # Se envía la lista de consultas utilizando la versión POST del recurso
        resp = requests.post(url, json=body)
        resp.raise_for_status()
        results = resp.json()['resultados']

        # Se crea un nuevo DataFrame utilizando el primer elemento del resultado de cada
        # consulta como filas
        tmp = pd.DataFrame(result[entity][0] if result[entity] else {} for result in results)
        normalized = normalized.append(tmp, ignore_index=True)

    normalized.index = data.index
    return pd.concat([normalized.add_prefix(prefix), data], axis='columns')

In [3]:
def georef_reverse_geocode(data, fields, params=None, prefix='gr_', step_size=1000):
    if not params:
        params = {}
        
    geocoded = pd.DataFrame()

    # Partir el DataFrame de entrada en partes de 'step_size' de longitud
    for i in range(0, len(data), step_size):
        end = min(len(data), i + step_size)
        queries = []

        for j in range(i, end):
            # Se crea cada 'consulta' individual (equivalente a hacer un request GET), y
            # se la agrega a una lista de consultas.
            query = {
                'aplanar': True,
                **params
            }

            for key, value in fields.items():
                query[key] = data.iloc[j][value]

            queries.append(query)

        body = {
            'ubicaciones': queries
        }

        # Se envía la lista de consultas utilizando la versión POST del recurso /ubicacion
        resp = requests.post('https://apis.datos.gob.ar/georef/api/ubicacion', json=body)
        resp.raise_for_status()
        results = resp.json()['resultados']

        # Se crea un nuevo DataFrame con los resultados de cada consulta como filas
        tmp =  pd.DataFrame(result['ubicacion'] for result in results).drop(columns=['lon', 'lat'])
        geocoded = geocoded.append(tmp)

    geocoded.index = data.index
    return pd.concat([geocoded.add_prefix(prefix), data], axis='columns')

#### A continuación, creamos la función que automatizará el proceso de normalización del dataset ventas y contempla la incorporación de deltas

In [5]:
def venta_normalizado(dataset):
    venta = pd.read_csv(dataset)

    'Observamos que tenemos un error de tipeo para las cantidades. Todos los outliers son múltiplos de 10. Se procede a llevar estas instancias a valores 1, 2 y 3'

    venta.loc[venta.Cantidad % 100 == 0,'Cantidad']= venta['Cantidad'] / 100
    venta.loc[venta.Cantidad % 10 == 0,'Cantidad']= venta['Cantidad'] / 10

    'Eliminamos outliers de precio'

    Q1 = venta['Precio'].quantile(0.25)
    Q3 = venta['Precio'].quantile(0.75)
    IQR = Q3 - Q1
    BI = Q1 - 1.5*IQR
    BS = Q3 + 1.5*IQR
    out_precio = (venta['Precio']<BI) | (venta['Precio']>BS) 

    venta = venta[~out_precio]

    'ACÁ SE TRABAJA CON LOS VALORES FALTANTES DE LA COLUMNA PRECIO y CANTIDAD'

    producto_venta = venta.dropna()
    producto_venta = producto_venta.drop_duplicates(subset = 'IdProducto')
    producto_venta  = producto_venta[['IdProducto', 'Precio']]
    venta_auxiliar = venta.copy()
    venta = venta_auxiliar.sort_values('IdVenta')

    'Imputamos valores faltantes de Cantidad con la moda, ya que no distorsiona nuestro gráfico de barras'

    moda = venta['Cantidad'].mode()
    moda = int(moda[0])
    venta['Cantidad'].fillna(value = moda, inplace = True)

    'Convertimos a entero, ya que estaba en decimal'

    venta['Cantidad'] = venta['Cantidad'].astype('int')

    'Convertimos los campos de fecha a su respectivo tipo'

    venta.Fecha = pd.to_datetime(venta.Fecha)
    venta.Fecha_Entrega = pd.to_datetime(venta.Fecha_Entrega)


    venta_original = pd.read_csv("Base_venta.csv", sep=",")
    venta_original.Fecha = pd.to_datetime(venta_original.Fecha)
    venta_original.Fecha_Entrega = pd.to_datetime(venta_original.Fecha_Entrega)


    veta_con_deltas = pd.concat([venta, venta_original], axis=0)
    veta_con_deltas = veta_con_deltas.sort_values(by=['IdVenta'])
    veta_con_deltas.to_csv("Base_venta.csv", index=False)

    return veta_con_deltas



In [6]:
venta_normalizado('Datasets\Venta.csv')

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2
4,5,2017-11-22,2017-11-25,2,678,13,1674,42825,554.18,3
...,...,...,...,...,...,...,...,...,...,...
46175,47596,2020-11-30,2020-12-09,1,193,26,3032,42850,279.0,2
46176,47597,2020-11-30,2020-12-07,3,333,26,3193,42799,1536.04,2
46177,47598,2020-11-30,2020-12-07,1,2389,27,3667,42916,1356.0,2
46178,47599,2020-11-30,2020-12-04,1,1067,29,3836,42935,2456.0,1


In [7]:
'Agregamos un delta al archivo original'

venta_normalizado('Datasets\Venta_Dic2020.csv')

,IdVenta,Fecha,Fecha_Entrega,IdCanal,IdCliente,IdSucursal,IdEmpleado,IdProducto,Precio,Cantidad
0,1,2018-03-09,2018-03-17,3,969,13,1674,42817,813.12,2
1,2,2018-12-28,2018-12-29,2,884,13,1674,42795,543.18,3
2,3,2016-03-28,2016-03-31,2,1722,13,1674,42837,430.32,1
3,4,2017-10-23,2017-10-24,3,2876,13,1674,42834,818.84,2
4,5,2017-11-22,2017-11-25,2,678,13,1674,42825,554.18,3
...,...,...,...,...,...,...,...,...,...,...
460,48237,2020-12-30,2021-01-02,1,1368,20,3186,42880,560.00,2
461,48238,2020-12-30,2021-01-02,2,354,23,3504,42883,498.00,1
462,48239,2020-12-30,2021-01-06,2,2400,25,3622,42814,369.82,2
463,48240,2020-12-30,2021-01-03,2,1803,30,3864,42998,529.00,3


### Aquí, se procede a crear la función que normalizará la tabla de compra

In [14]:
def compra_normalizada(dataset):
    compra = pd.read_csv(dataset)

    'Eliminamos la columnma que no usaremos'

    compra.drop(columns="Fecha_Periodo", inplace=True)

    'Convertimos fecha a datetime'

    compra.Fecha = pd.to_datetime(compra.Fecha)

    'Realizamos la búsqueda de outliers de precio y cantidad para. posteriormente. eliminarlos'
    
    Q1 = compra['Precio'].quantile(0.25)
    Q3 = compra['Precio'].quantile(0.75)
    IQR = Q3 - Q1
    BI = Q1 - 1.5*IQR
    BS = Q3 + 1.5*IQR
    out_precio = (compra['Precio']<BI) | (compra['Precio']>BS)

    Q1 = compra['Cantidad'].quantile(0.25)
    Q3 = compra['Cantidad'].quantile(0.75)
    IQR = Q3 - Q1
    BI = Q1 - 1.5*IQR
    BS = Q3 + 1.5*IQR
    out_cantidad = (compra['Cantidad']<BI) | (compra['Cantidad']>BS)

    compra = compra[~out_cantidad]
    compra[~out_precio]

    'Resetamos índice'

    compra.reset_index(inplace= True, drop = True)

    'En esta etapa, realizamos el tratamiento de los valores faltantes'

    producto_compra = compra.dropna()
    producto_compra  = producto_compra[['IdProducto', 'Precio']]
    producto_compra_unicos = producto_compra['IdProducto'].drop_duplicates()
    producto_compra_unicos.reset_index(inplace= True, drop = True)
    producto_compra_unicos = pd.DataFrame(producto_compra_unicos)

    'Nos traemos el promedio de los precios de los IdProductos únicos para luego imputar'
    for i in range(0, producto_compra_unicos.shape[0]):
        idproducto = producto_compra_unicos.iloc[i][0]
        precio_promedio = producto_compra['Precio'][producto_compra['IdProducto'] == idproducto].mean()
        producto_compra_unicos.loc[producto_compra_unicos['IdProducto'] == idproducto,'Precio_promedio'] = precio_promedio   

    compra = compra.merge(producto_compra_unicos, how='inner', on='IdProducto')

    compra.Precio.fillna('SinDato', inplace = True)
    compra.Precio.replace('SinDato', precio_promedio, inplace= True)
    compra.drop(columns = 'Precio_promedio', inplace = True)

    'Finalmemte, guardamos la tabla normalizada en un csv contemplando los deltas'

    compra_original = pd.read_csv("Base_compra.csv", sep=",")
    compra_original.drop(columns="Fecha_Periodo", inplace=True)

    compra_con_deltas = pd.concat([compra, compra_original], axis=0)
    compra_con_deltas = compra_con_deltas.sort_values(by=['IdCompra'])
    compra_con_deltas.Fecha = pd.to_datetime(compra_con_deltas.Fecha)

    compra_con_deltas.to_csv("Base_compra.csv", index=False)

    return compra_con_deltas

In [15]:
compra_normalizada('Datasets\Compra.csv')

C:\Users\leand\AppData\Local\Temp\ipykernel_42960\3116515441.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  compra[~out_precio]


,IdCompra,Fecha,Fecha_Año,Fecha_Mes,IdProducto,Cantidad,Precio,IdProveedor
0,1,2015-01-30,2015,1,42832,13,560.51,12
32,2,2015-01-30,2015,1,42833,11,497.58,7
77,3,2015-01-30,2015,1,42834,1,588.5,6
124,4,2015-01-30,2015,1,42835,9,567.66,14
156,5,2015-01-30,2015,1,42839,14,231.31,2
...,...,...,...,...,...,...,...,...
5719,11535,2020-12-25,2020,12,42978,6,88.29,10
5761,11536,2020-12-25,2020,12,42979,2,81.24,10
5800,11537,2020-12-25,2020,12,42980,1,71.67,3
10153,11538,2020-12-25,2020,12,42981,3,35.55,9


In [9]:
def gasto_normalizado(dataset):

    'Leemos el dataset'
    gasto = pd.read_csv(dataset)

    'Leemos una primera tabla de gasto que solo incorpore los encabezadost'

    gasto_original = pd.read_csv("Base_gasto.csv", sep=",")

    'Realizamos la concatenación para futuros deltas'
    gasto_con_deltas = pd.concat([gasto, gasto_original], axis=0)
    gasto_con_deltas = gasto_con_deltas.sort_values(by=['IdGasto'])
    gasto_con_deltas.to_csv("Base_gasto.csv", index=False)

    return gasto_con_deltas

In [11]:
gasto_normalizado('Datasets\Gasto.csv')

,IdGasto,IdSucursal,IdTipoGasto,Fecha,Monto
0,1,18,1,2015-01-01,1164.9
1,2,18,2,2015-01-01,317.02
2,3,18,3,2015-01-01,118.89
3,4,18,4,2015-01-01,1174.23
4,5,1,1,2015-01-01,1104.51
...,...,...,...,...,...
8635,8636,23,4,2020-12-01,1180.27
8636,8637,24,1,2020-12-01,1152.15
8637,8638,24,2,2020-12-01,305.98
8638,8639,24,3,2020-12-01,104.32


#### En este momento, se crea la función que automatizará el proceso de normalización del dataset clientes y contempla la incorporación de deltas

In [12]:
def cliente_normalizada(dataset):
    cliente = pd.read_csv(dataset, sep = ';', encoding = 'utf-8')

    'Eliminamos la columna que no vamos a utilizar'

    cliente.drop(['col10'], axis = 1, inplace=True)

    ' Renombramos las columnas'

    cliente.Nombre_y_Apellido = cliente.Nombre_y_Apellido.str.title()
    cliente.Provincia = cliente.Provincia.str.title()
    cliente.Localidad = cliente.Localidad.str.title()
    cliente.Domicilio = cliente.Domicilio.str.title()

    cliente.rename(columns={'ID':'IdCliente'}, inplace=True)
    cliente.rename(columns={'X':'Longitud'}, inplace=True)
    cliente.rename(columns={'Y':'Latitud'}, inplace=True)

    ' Pasamos a mayúscula inicial'

    'Casteamos y reemplazamos comas por puntos'

    cliente['Longitud'] = cliente['Longitud'].str.replace(",", ".").astype(float)
    cliente['Latitud'] = cliente['Latitud'].str.replace(",", ".").astype(float)
    

    'Reemplazamos aquellas latitudes y longitudes que estén en positivo'

    for index, row in cliente['Longitud'].iteritems():
        if row > 0:
            cliente['Longitud'][index] = row * (-1)

    for index, row in cliente['Latitud'].iteritems():
        if row > 0:
            cliente['Latitud'][index] = row * (-1)
    
    'Acá procedemos a reemplazar latitud por longitud cuando están mal, creando una columna auxiliar'

    cliente.loc[cliente.Latitud < -57,'new_Longitud']= cliente.Latitud
    cliente.loc[cliente.new_Longitud < -57, 'Latitud'] = cliente.Longitud
    cliente.loc[cliente.new_Longitud < -57, 'Longitud'] = cliente.new_Longitud

    cliente.drop(['new_Longitud'], axis = 1, inplace = True)


    'Separamos los valores faltantes del dataset en latitud, longitud, localidad y provincia para poder trabajar con las funciones provistas por la API'
    
    cliente.fillna('SinDato', inplace=True)

    faltante_longitud = cliente['Longitud']  == 'SinDato'
    df_faltante_longitud = cliente.loc[faltante_longitud]
    cliente = cliente[~faltante_longitud]

    faltante_latitud = cliente['Latitud']  == 'SinDato'
    df_faltante_latitud = cliente.loc[faltante_latitud]
    cliente = cliente[~faltante_latitud]

    clientes_faltantes = pd.concat([df_faltante_longitud, df_faltante_latitud], axis=0)
    clientes_faltantes.drop_duplicates(inplace=True)

    clientes_faltantes = georef_normalize_entities(clientes_faltantes, 'localidades', {'nombre': 'Localidad'})
    clientes_faltantes.drop(columns={'gr_categoria', 'gr_departamento_id', 'gr_id', 'gr_localidad_censal_id', 'Longitud', 'Latitud', 'gr_localidad_censal_nombre', 'gr_municipio_id', 'Localidad', 'gr_provincia_nombre', 'gr_provincia_id', 'gr_municipio_nombre', 'gr_departamento_nombre', 'Localidad'}, inplace=True)
    clientes_faltantes.rename(columns={'gr_centroide_lat':'Latitud', 'gr_centroide_lon':'Longitud', 'gr_nombre':'Localidad'}, inplace=True)
    clientes_faltantes = clientes_faltantes.reindex(columns=['IdCliente', 'Provincia', 'Nombre_y_Apellido', 'Domicilio', 'Telefono', 'Edad', 'Localidad', 'Longitud', 'Latitud'])

    faltante_provincia = cliente['Provincia']  == 'SinDato'
    df_faltante_provincia = cliente.loc[faltante_provincia]
    cliente = cliente[~faltante_provincia]

    faltante_localidad = cliente['Localidad']  == 'SinDato'
    df_faltante_localidad = cliente.loc[faltante_localidad]
    cliente = cliente[~faltante_localidad]

    clientes_faltantes_2 = pd.concat([df_faltante_provincia, df_faltante_localidad], axis=0)
    clientes_faltantes_2.drop_duplicates(inplace=True)


    clientes_faltantes_2 = georef_reverse_geocode(clientes_faltantes_2, {'lat': 'Latitud', 'lon': 'Longitud'})
    clientes_faltantes_2.drop(columns={'gr_departamento_id','gr_departamento_nombre','gr_municipio_id', 'gr_municipio_nombre', 'gr_provincia_id', 'Provincia'}, inplace=True)
    clientes_faltantes_2.rename(columns={'gr_provincia_nombre':'Provincia'}, inplace=True)
    clientes_faltantes_2 = clientes_faltantes_2.reindex(columns=['IdCliente', 'Provincia', 'Nombre_y_Apellido', 'Domicilio', 'Telefono', 'Edad', 'Localidad', 'Longitud', 'Latitud'])

    cliente_final = pd.concat([clientes_faltantes, clientes_faltantes_2, cliente], axis=0)
   
    'Aquí, traemos el csv original'
    cliente_original = pd.read_csv('Base_cliente.csv', sep=',')
    cliente_original.rename(columns={'ID':'IdCliente', 'X': 'Longitud', 'Y': 'Latitud'}, inplace=True)

    'Incorporamos los deltas'

    clientes_con_deltas = pd.concat([cliente_final, cliente_original], axis=0)
    clientes_con_deltas = clientes_con_deltas.sort_values(by=['IdCliente'])


    'Exportamos'
    
    clientes_con_deltas.to_csv("Base_cliente.csv", index=False)


    return clientes_con_deltas

In [13]:
cliente_normalizada('Datasets\Clientes.csv')

C:\Users\leand\AppData\Local\Temp\ipykernel_49624\2369141283.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cliente['Longitud'][index] = row * (-1)
C:\Users\leand\AppData\Local\Temp\ipykernel_49624\2369141283.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cliente['Latitud'][index] = row * (-1)
C:\Users\leand\AppData\Local\Temp\ipykernel_49624\1267500554.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normalized = normalized.append(tmp, ignore_index=True)
C:\Users\leand\AppData\Local\Temp\ipykernel_49624\498338858.p

,IdCliente,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,Longitud,Latitud
0,1,Buenos Aires,Heber Joni Santana,Las Heras Y Bat. 24 De Febrero 4150 Rincon De...,42-5161,58,Loma Verde,-58.818503,-34.309971
1,2,Buenos Aires,Ana Sapriza,Pueyrredon Y Dupuy Ruta3 Km 52.500 S/N Bâº Lo...,49-7578,61,Santa Rosa,-58.730738,-34.939083
2,3,Buenos Aires,Fernando Luis Saralegui,Calderon De La Barca 498,49-3435,15,Torres,-59.127941,-34.430822
3,4,Buenos Aires,Manuela Sarasola,"Ruta 36 Km 45,500 S/N El Peligro",49-2883,29,Ruta Sol,-58.14394,-34.920527
4,5,Buenos Aires,Mario Raúl Sarasua,492 Y 186 S/N Colonia Urquiza,491-4608,34,Jose Melchor Romero,-58.089381,-34.944447
...,...,...,...,...,...,...,...,...,...
2667,2668,Córdoba,Maria Stella Rivas,Calle Publica Santa Elena Colonia Tirolesa - ...,4902756,23,Colonia Tirolesa,-64.074193,-31.258817
2668,2669,Córdoba,Ana Gabriela Rivas,Calle Publica S/Nâº La Arcadia,153580992,28,La Arcadia,-64.076969,-31.533384
2669,2670,Córdoba,Javier Alonso Riveiro,Avenida Libertador San Martin S/Nâº Bouwer,4993041,57,Bouwer,-64.192983,-31.557861
2670,2671,Córdoba,Juan Manuel Rodriguez,Calle Publica S/Nâ° El Gateado Barrio El Gate...,152768918,36,El Gateado,-64.116812,-31.321234


In [14]:
'Realizamos un delta'

cliente_normalizada("Datasets\Clientes_v2.csv")

C:\Users\leand\AppData\Local\Temp\ipykernel_49624\1267500554.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normalized = normalized.append(tmp, ignore_index=True)
C:\Users\leand\AppData\Local\Temp\ipykernel_49624\498338858.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geocoded = geocoded.append(tmp)


,IdCliente,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,Longitud,Latitud
0,1,Buenos Aires,Heber Joni Santana,Las Heras Y Bat. 24 De Febrero 4150 Rincon De...,42-5161,58,Loma Verde,-58.818503,-34.309971
1,2,Buenos Aires,Ana Sapriza,Pueyrredon Y Dupuy Ruta3 Km 52.500 S/N Bâº Lo...,49-7578,61,Santa Rosa,-58.730738,-34.939083
2,3,Buenos Aires,Fernando Luis Saralegui,Calderon De La Barca 498,49-3435,15,Torres,-59.127941,-34.430822
3,4,Buenos Aires,Manuela Sarasola,"Ruta 36 Km 45,500 S/N El Peligro",49-2883,29,Ruta Sol,-58.14394,-34.920527
4,5,Buenos Aires,Mario Raúl Sarasua,492 Y 186 S/N Colonia Urquiza,491-4608,34,Jose Melchor Romero,-58.089381,-34.944447
...,...,...,...,...,...,...,...,...,...
730,3403,Tucumán,Edmundo Roselli,Camino De Las Lanzas Y San Martin,SinDato,64,Cañada De Yerba Buena,-65.296801,-26.832451
731,3404,Tucumán,Gerardo Manuel Rosendo,Ruta 306- Km. 07 Pacara Pintado,SinDato,26,Pacara Pintado,-65.181107,-26.882212
732,3405,Tucumán,Jose Luis Ruocco,El Talar,4301318,21,El Talar,-65.108517,-26.802963
733,3406,Tucumán,Ana Maria Salveraglio,Gral. San Martin Manzana 3,SinDato,62,Los Nogales,-65.218737,-26.712922


### Ahora se avanza en la normalización de la tabla de proveedores

In [17]:
def proveedor_normalizada(dataset):
    
    proveedor = pd.read_csv(dataset, sep= ',', encoding= 'latin-1')
    
    'Utilizamos la API'

    proveedor = georef_normalize_entities(proveedor, 'departamentos', {'nombre': 'departamen'})
    
    'Nos quedamos con aquellas columnas que nos interesan'
    
    proveedor.drop(columns= ['Country', 'gr_centroide_lat', 'gr_centroide_lon', 'gr_id', 'gr_provincia_id', 'departamen', 'gr_provincia_nombre'], inplace=True)
    proveedor.rename(columns= {"gr_nombre":"Departamento", "Address":"Dirección", "State":"Provincia", 'City':'Localidad', 'IDProveedor' : 'IdProveedor'}
    , inplace=True)

    
    proveedor = georef_normalize_entities(proveedor, 'provincias', {'nombre': 'Provincia'})
    proveedor.drop(columns=["gr_centroide_lat", "gr_centroide_lon", "gr_id", "Provincia"])
    proveedor = proveedor.reindex(columns=['IdProveedor', 'Nombre', 'Provincia', 'Departamento', 'Localidad', 'Dirección'])
    
    
    'Reenombraos'

    proveedor.rename(columns={"gr_nombre":"Provincia"}, inplace=True)    
    proveedor.drop_duplicates(inplace=True)

    'Colocamos correctamente cada campo con mayúsculas inciales'

    proveedor.Provincia = proveedor.Provincia.str.title()
    proveedor.Localidad = proveedor.Localidad.str.title()
    proveedor.Dirección = proveedor.Dirección.str.title()
    
    
    'Exportamos'

    proveedor.to_csv('Proveedor_normalizada.csv', index=False)

    return proveedor

In [19]:
proveedor_normalizada('Datasets\Proveedores.csv')

C:\Users\leand\AppData\Local\Temp\ipykernel_49624\1267500554.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normalized = normalized.append(tmp, ignore_index=True)
C:\Users\leand\AppData\Local\Temp\ipykernel_49624\1267500554.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  normalized = normalized.append(tmp, ignore_index=True)


,IdProveedor,Nombre,Provincia,Departamento,Localidad,Dirección
0,1,NaN,Buenos Aires,Avellaneda,Avellaneda,Bereguestein 36
1,2,San Cirano,Buenos Aires,Brandsen,Coronel Brandsen,Alberti 965
2,3,Bell S.A.,Buenos Aires,Campana,Campana,Moreno 331
3,4,Rivero Insumos,Buenos Aires,Lanús,Valentin Alsina,Pte. Peron 3872
4,5,Laprida Computacion,Buenos Aires,Laprida,Laprida,Avda. Pedro Pereyra 1339
5,6,Importadora Mann Kloss,Caba,Comuna 10,Boca De Atencion Monte Castro,Lope De Vega 2102
6,7,Fletes y Logistica,Cordoba,Capital,Cordoba,Santa Rosa 1564
7,8,NaN,Cordoba,Capital,Cordoba,Santa Rosa 1564
8,9,Via Chile Containers,Mendoza,Capital,Mendoza,Juan B. Justo 415
9,10,Full Toner,Mendoza,Capital,Mendoza,Cordoba 246


### Por último, normalizamos el dataset de sucursales

In [44]:
def sucursal_normalizada(dataset):
    
    sucursal = pd.read_csv(dataset, delimiter = ';')
   
    sucursal.rename(columns={'ID':'IdSucursal'}, inplace=True)
    sucursal['Latitud'] = sucursal['Latitud'].str.replace(",", ".").astype(float)
    sucursal["Longitud"] = sucursal["Longitud"].str.replace(",", ".").astype(float)

    sucursal = georef_reverse_geocode(sucursal, {'lat': 'Latitud', 'lon': 'Longitud'})
    
    sucursal.drop(["gr_departamento_id", "gr_municipio_id", "gr_municipio_nombre", "gr_provincia_id"], axis=1, inplace=True)
    sucursal.drop(columns="Provincia", inplace=True)
    sucursal.rename(columns= {"gr_departamento_nombre":"Departamento", "gr_provincia_nombre":"Provincia"}, inplace=True)
    #sucursal.drop(columns="Localidad", inplace=True)
    
    
    #sucursal.to_csv("SucursalNOR.csv", index=False)



    'Colocamos correctamente cada campo con mayúsculas inciales'

    sucursal.Provincia = sucursal.Provincia.str.title()
    sucursal.Localidad = sucursal.Localidad.str.title()
    sucursal.Direccion = sucursal.Direccion.str.title()
    
    sucursal = sucursal.reindex(columns=['IdSucursal', 'Sucursal', 'Provincia', 'Departamento', 'Direccion','Longitud', 'Latitud'])    
    'Exportamos el dataset normalizado'

    sucursal.to_csv("Sucursal_normalizada.csv", index=False)

    return sucursal

In [45]:
sucursal_normalizada('Datasets\Sucursales.csv')

C:\Users\leand\AppData\Local\Temp\ipykernel_49624\498338858.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geocoded = geocoded.append(tmp)


,IdSucursal,Sucursal,Provincia,Departamento,Direccion,Longitud,Latitud
0,1,Cabildo,Ciudad Autónoma De Buenos Aires,Comuna 13,Av. Cabildo 1342,-58.449572,-34.567806
1,2,Palermo 1,Ciudad Autónoma De Buenos Aires,Comuna 14,Guatemala 5701,-58.433566,-34.579035
2,3,Palermo 2,Ciudad Autónoma De Buenos Aires,Comuna 2,Gral. Lucio Norberto Mansilla 2668,-58.405150,-34.595966
3,4,Corrientes,Ciudad Autónoma De Buenos Aires,Comuna 3,Av. Corrientes 2352,-58.398764,-34.604685
4,5,Almagro,Ciudad Autónoma De Buenos Aires,Comuna 5,Venezuela 3650,-58.416179,-34.617308
5,6,Caballito,Ciudad Autónoma De Buenos Aires,Comuna 6,Av. Rivadavia 4708,-58.431849,-34.616303
6,7,Flores,Ciudad Autónoma De Buenos Aires,Comuna 6,Av. Rivadavia 5746,-58.446449,-34.622868
7,8,Alberdi,Ciudad Autónoma De Buenos Aires,Comuna 6,Av. Juan Bautista Alberdi 1634,-58.451498,-34.627306
8,9,Deposito,Ciudad Autónoma De Buenos Aires,Comuna 7,Pedernera 530,-58.461513,-34.635134
9,10,Velez,Ciudad Autónoma De Buenos Aires,Comuna 10,Av. Juan Bautista Justo 7738,-58.498063,-34.631257
